In [1]:
import os
os.chdir("../")

In [2]:
# imports
import warnings
warnings.filterwarnings("ignore")

import time
import openai
import json
import numpy as np
import pandas as pd
from datasets import Dataset, DatasetDict
from mega.data.data_utils import choose_few_shot_examples
from mega.prompting.instructions import INSTRUCTIONS
from mega.prompting.prompting_utils import load_prompt_template
from mega.utils.env_utils import load_env
from mega.models.completion_models import get_model_pred, gpt3x_completion
from mega.prompting.prompting_utils import construct_prompt, construct_cmxnli_prompt
from tqdm import tqdm

In [3]:
# Make sure that {env_name}.env file is present in the envs/ directory
env_name = "melange"
load_env(env_name=env_name)

In [4]:
openai.api_base

'https://gpttesting1.openai.azure.com/'

In [5]:
model = "gpt-35-turbo-deployment"
prompt_name = "GPT-3 style"
few_shot_k = 8

In [6]:
# Read the extracted JSON file
with open('gluecosdata/nli/all.json', 'r') as json_file:
    # Load the JSON data
    data = json.load(json_file)
    
    # Convert the JSON data to a pandas DataFrame
    df = pd.DataFrame(data)
    
    # Read the text file with test set IDs
    with open('gluecosdata/nli/test_ids.txt', 'r') as test_ids_file:
        # Extract the IDs as a list of strings
        test_ids = [int(line.strip()) for line in test_ids_file]
        
        # Split the DataFrame into train and test subsets based on the IDs
        train_df = df[~df['ID'].isin(test_ids)].reset_index(drop=True)
        test_df = df[df['ID'].isin(test_ids)].reset_index(drop=True)
        
        # Convert the train and test DataFrames to Dataset objects
        train_dataset = Dataset.from_pandas(train_df)
        test_dataset = Dataset.from_pandas(test_df)
        
        # Create a DatasetDict object with train and test datasets
        dataset_dict = DatasetDict({'train': train_dataset, 'test': test_dataset})
        
    # Close the test IDs file
    test_ids_file.close()

# Close the JSON file
json_file.close()

In [7]:
train_dataset[0]

{'ID': 0,
 'Premise ID': '465',
 'Premise': 'FATHER\nMain wapis hospital jaa raha hun .\nRAHUL\nMaa , main dieting pe hun , mere liye green tea please .\nMOTHER\nHaan , okay beta .\nRAHUL\nEk aur baat .. (he pulls something out of his pocket) Ye aapke liye ..\nMOTHER\nOh .. You are just .. Mera perfect baccha .\n',
 'Hypothesis': 'Dady wapas hospital ja rahe hain',
 'Label': 'entailed'}

In [8]:
train_dataset["Premise"][0]

'FATHER\nMain wapis hospital jaa raha hun .\nRAHUL\nMaa , main dieting pe hun , mere liye green tea please .\nMOTHER\nHaan , okay beta .\nRAHUL\nEk aur baat .. (he pulls something out of his pocket) Ye aapke liye ..\nMOTHER\nOh .. You are just .. Mera perfect baccha .\n'

In [9]:
template = """{premise} Question: {hypothesis} True or False? ||| {label}"""
verbalizer = { "entailed": "True", "contradiction": "False"}

In [10]:
# Loading instruction for the task
instruction = INSTRUCTIONS["xnli"]
print(instruction)

You are an NLP assistant whose purpose is to solve Natural Language Inference (NLI) problems. NLI is the task of determining the inference relation between two (short, ordered) texts: entailment, contradiction, or neutral. Answer as concisely as possible in the same format as the examples below:


In [11]:
# Getting few-shot examples
train_examples = choose_few_shot_examples(
        train_dataset, few_shot_k, selection_criteria="random")

In [12]:
test_example = test_dataset[0]

prompt, label = construct_cmxnli_prompt(
    train_examples,
    test_dataset[0],
    train_prompt_template=template,
    test_prompt_template=template,
    chat_prompt=False,
    instruction=instruction,
    verbalizer=verbalizer
)
prompt

"SARA\nMei n t oh rah oon gi na hin , l eki n tumhare liye ek aur nominee hota toh accha hota ..\nDEEPAK\nHaan achha hota ..\nSARA\nSoch lo phi r ... ready hoon main abhi bhi !\nDEEPAK\nSh ay a d p ro bl e m m ei n h ai n v o h ! Pa re s ha a n th e in us s d in su b ah subah !\nSARA\nKaun ?\nDEEPAK\nVoh B 48 . 1 st floor . B ataya tha na ...\nSARA\nVoh teenon bachiyan ...\n Question: usne B 48 1st floor bataya tha True or False? |||\nTrue\nDR. SWAMY\nYou know the law Ashwin .. Investigative officers narco test mein maujud nahin reh sakte ..\nASHWIN\nSwamy Saab .. Main pahadi ki nok pe khada hoon .. Ek kood mili .. Aur khai paar .. Please ..\nDR. SWAMY\nAshwin bachhe ki tarah zid mat karo .. I can't help in this , please ..\nASHWIN\nAchha phone do apna .\nDR. SWAMY\nKisey phone laga rahe ho ?\nASHWIN\nKhud ko ? (He gives her the phone back\nLo baat karo .\nASHWIN\nHello Dr . Swamy ? Kaisi hain ?\nDR. SWAMY\nAshwin kya hai ye ?\nASHWIN\nLive telecast . Ye to law mein mana nahi hai na ? 

In [13]:
prediction = gpt3x_completion(
    prompt,
    model,
    temperature=0,
    max_tokens=10
)
match = float(prediction.startswith(label))
print(f"Prediction: {prediction}")
print(f"Label: {label}")
print(f"Match: {match}")

Prediction: False
Label: True
Match: 0.0


In [14]:
matches = []
preds = []
labels = []
for test_example in tqdm(test_dataset):
    prompt, label = construct_cmxnli_prompt(
        train_examples,
        test_example,
        train_prompt_template=template,
        test_prompt_template=template,
        chat_prompt=False,
        instruction=instruction,
        verbalizer=verbalizer
    )
    prediction = gpt3x_completion(
        prompt,
        model,
        temperature=0,
        max_tokens=10
    )
    time.sleep(1/2)
    match = float(prediction.startswith(label))
    preds.append(prediction)
    labels.append(label)
    matches.append(match)

print(f"Accuracy: {np.mean(matches)}")

100%|██████████| 448/448 [11:58<00:00,  1.60s/it]

Accuracy: 0.7209821428571429
